In [204]:
#importamos la base de datos
import pandas as pd #pandas
df=pd.read_csv('airbnb.csv',encoding='latin-1')
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [205]:
#importamos todas las librerías necesarias: 
import sklearn #scikit-learn
import matplotlib.pyplot as plt #gráfica Matplotlib
import numpy as np #NumPy

Primero debemos ver qué hacemos con los datos NaN para tratarlos

Analizamos cuántos datos NaN hay en la columna reviews_per_month


In [206]:
reviews_per_month_nan = df['reviews_per_month'].isna().sum()
print(reviews_per_month_nan)

10052


Ahora bien, vemos que hay 10052 datos NaN en la columna. Observemos cuántos datos hay en total para barajar las distintas opciones de qué hacer con estos datos NaN. Si es poca cantidad comparado con la cantidad de observaciones en total, podremos eliminar directamente esas observaciones, sin embargo, si suponen gran cantidad, podríamos buscar otras opciones, como rellenar con la media para así no afectar al equilibrio del resto de datos. 
También es verdad que como hablamos de reseñas por mes, sería factible convertir esos NaN en 0, para que así a la hora de categorizar la columna, lo detecte como Bajo. 

In [207]:
num_observations = df.shape[0]
print(num_observations)

48895


Supone un 20% de los datos totales. No resulta útil borrarlos, si no que deberíamos rellenar con 0's. 

In [208]:
df['reviews_per_month'].fillna(0, inplace=True)

¿Por qué rellenamos con 0's? 
Encontramos un NaN en esa columna, cuando en la columna de number_of_reviews aparece un 0, o un NaN directamente, por lo que no sería "justo" por así decirlo rellenar con la media, ya que ahí estarías metiendo posibles "evaluaciones" que no existen. 

In [209]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,0.00,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


Comprobamos por tanto que en esa columna ya no nos quede ningún NaN

In [210]:
reviews_per_month_nan = df['reviews_per_month'].isna().sum()
print(reviews_per_month_nan)

0


Ya nos hemos asegurado que no nos quedad ningún dato NaN, así no encontraremos ningún error a la hora de categorizar la columna. 

Si no también otra manera de hacerlo es: 
calculando a mano 3 rangos, ver cuál es el mayor, el menos y dividir en 3 rangos. 


-Crea una lista de intervalos de tres categorías

bins = [0, 3, 6, np.inf]

es un ejemplo de los rangos, los debemos calcular bien. 

-Asigna una categoría a cada valor en la columna 'reviews_per_month'

df['reviews_per_month'] = pd.cut(df['reviews_per_month'], bins, labels=['Baja', 'Media', 'Alta'])

In [211]:
df['reviews_per_month'].max()

58.5

In [212]:
index_max = df['reviews_per_month'].idxmax()
print(index_max)

42075


In [213]:
print(df['reviews_per_month'].head(index_max+1))

0         0.21
1         0.38
2         0.00
3         4.64
4         0.10
         ...  
42071     1.69
42072     0.68
42073     7.08
42074     0.00
42075    58.50
Name: reviews_per_month, Length: 42076, dtype: float64


In [214]:
# Define el número de intervalos
num_intervalos = 3

# Crea una lista de intervalos
intervalos = np.linspace(df['reviews_per_month'].min()-0.001, df['reviews_per_month'].max(), num_intervalos+1)

# Creamos las etiquetas 
etiquetas = ['Baja', 'Media', 'Alta']

# Asigna una categoría a cada intervalo
df['reviews_per_month'] = pd.cut(df['reviews_per_month'], intervalos, labels=etiquetas)

In [215]:
print(df['reviews_per_month'].head(index_max+1))

0        Baja
1        Baja
2        Baja
3        Baja
4        Baja
         ... 
42071    Baja
42072    Baja
42073    Baja
42074    Baja
42075    Alta
Name: reviews_per_month, Length: 42076, dtype: category
Categories (3, object): ['Baja' < 'Media' < 'Alta']


In [216]:
#aqui podemos contar cuanto hay de cada tipo para ampliar un poco

Por otro lado, vamos a clasificar los alquileres en función de los días que están disponibles. 

Para ello, siguiendo el patrón anterior, calculamos si hay NaN en la columna correspondiente: 


In [217]:
disponibilidad_nan = df['availability_365'].isna().sum()
print(disponibilidad_nan)

0


No hay ningún NaN en dicha columna, por lo que no necesitamos reemplazar los datos por 0 en ningún alquiler. 

Calculemos los intervalos de los rangos. 

In [218]:
df['availability_365'].max()

365

In [219]:
i_max = df['availability_365'].idxmax()
i_max

0

In [220]:
print(df['availability_365'].head(i_max+1))

0    365
Name: availability_365, dtype: int64


In [221]:
#Definimos los intervalos
n_inter = 3

# Creamos una lista de intervalos
interv = np.linspace(df['availability_365'].min()-0.001, df['availability_365'].max(), n_inter+1)

# Creamos las etiquetas 
labels = ['Poco disponible', 'Disponible normalmente', 'Altamente disponible']

# Asigna una categoría a cada intervalo
df['availability_365'] = pd.cut(df['availability_365'], interv, labels=labels)

In [222]:
print(df['availability_365'].head())

0      Altamente disponible
1      Altamente disponible
2      Altamente disponible
3    Disponible normalmente
4           Poco disponible
Name: availability_365, dtype: category
Categories (3, object): ['Poco disponible' < 'Disponible normalmente' < 'Altamente disponible']


Finalmente, añadimos la nueva columna de clasificación: 

In [225]:
df.loc[:,'classification']=np.nan
print(df)

             id                                               name   host_id  \
0          2539                 Clean & quiet apt home by the park      2787   
1          2595                              Skylit Midtown Castle      2845   
2          3647                THE VILLAGE OF HARLEM....NEW YORK !      4632   
3          3831                    Cozy Entire Floor of Brownstone      4869   
4          5022   Entire Apt: Spacious Studio/Loft by central park      7192   
...         ...                                                ...       ...   
48890  36484665    Charming one bedroom - newly renovated rowhouse   8232441   
48891  36485057      Affordable room in Bushwick/East Williamsburg   6570630   
48892  36485431            Sunny Studio at Historical Neighborhood  23492952   
48893  36485609               43rd St. Time Square-cozy single bed  30985759   
48894  36487245  Trendy duplex in the very heart of Hell's Kitchen  68119814   

           host_name neighbourhood_grou

La creamos entera con NaN que ahora modificaremos según los dos valores que hemos clasifiacado anteriormente.

tabla

In [253]:

filas=len(df.axes[0])
print(filas)
for i in range (filas):
  if df['availability_365'][i]=='Poco disponible':
    if df['reviews_per_month'][i]=='Alta':
      df['classification'][i]='Muy bueno'
    else:
      df['classification'][i]='Bueno'
  elif df['availability_365'][i]=='Disponible normalmente':
    if df['reviews_per_month'][i]=='Alta':
      df['classification'][i]='Bueno'
    else:
      df['classification'][i]='Regular'
  else:
    if df['reviews_per_month'][i]=='Alta':
      df['classification'][i]='Muy malo'
    else:
      df['classification'][i]='Malo'


48895


Comprobemos que no quede ningun NaN:

In [254]:
classif_nan = df['classification'].isna().sum()
print(classif_nan)

0


Todos los datos de la nueva columna han sido modificados, por lo que ya tenemos clasificados los alquieleres. 